# Screening Task 2

### Here I've implemented a circuit that returns |01> and |10> with equal probabilities

* The circuit consists of CNOT, RX and RY gates
* I've started the all initial parameters being in randomly chosen state
* I've used Pennylane-Qiskit plugin to simulate the noise model of ibmq_ourense quantum computer



In [1]:
import pennylane as qml
from pennylane import numpy as np
import qiskit
from qiskit.providers.aer.noise.device import basic_device_noise_model


In [2]:
qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider(group='open')
ibmq_16_melbourne = provider.get_backend('ibmq_ourense')
device_properties = ibmq_16_melbourne.properties()

noise_model = basic_device_noise_model(device_properties)

/Users/adamdavis/miniconda3/envs/quantum/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
/Users/adamdavis/miniconda3/envs/quantum/lib/python3.8/site-packages/qiskit/providers/aer/noise/device/basic_device_model.py:111: DeprecationWarning: This function is been deprecated and moved to a method of the`NoiseModel` class. For equivalent functionality use `NoiseModel.from_backend(properties, **kwargs).
  warnings.warn(


In [3]:
dev1 = qml.device('qiskit.aer', wires=2, backend='qasm_simulator',noise_model=noise_model)

In [4]:
@qml.qnode(dev1)
def circuit(params):
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RX(params[2], wires=0)
    qml.CNOT(wires=[0,1])
    return qml.probs(wires=[0, 1])

* A brief point to note here is that, since we are using a noise model, it is impossible to get a  perfect probabilites of   **[0 0.5 0.5 0]**, where this **list represents the probabilities of the states|00>, |01>, |10>, |11>** respectively, and this is the required list

* The required list of parameters for obtaining our circuit is therefore [$\pi/2$ $\pi$ $\pi$]



In [6]:
print(circuit([np.pi/2,np.pi,np.pi]))


[0.07324219 0.48632812 0.42480469 0.015625  ]


In [7]:
print(circuit.draw())

 0: ──RY(1.571)──RX(3.142)──╭C──╭┤ Probs 
 1: ──RY(3.142)─────────────╰X──╰┤ Probs 



## The Cost function

#### I've used a cost function that measures the difference between the current list of probabilities and the required list of probablities of **[0 0.5 0.5 0]**

* The cost function I've defined below takes the current list of probabilities, gets the second and third item of the list, subtracts 0.5 from it, squares and adds them ,and then performs a square root

* The reason I'm using such a cost function is because this function highly penalizes the list of probabilities if they are very far from the required list **[0 0.5 0.5 0]**, and we can see our gradient descent optimiser learning slowly to get us close to the required parameters

* If our given list of probabilities is y, then the cost function is $$\sqrt{(y[1]-0.5)^2 + (y[2]-0.5)^2)}$$


In [8]:
def cost(x):
    k= ((circuit(x)[1]-0.5)**2+(circuit(x)[2]-0.5)**2)**(1/2)
    return k
    


Here I've chosen the initial parameters [0.001 0.004 0.003] randomly, it __does not mean__ that there are certain magical properties in these initial values. They are chosen absolutely randomly chosen.

In [14]:
init_params = np.array([0.001,0.004,0.003]) # I have selected random values here, not that 
# initial values have helped me reduce the initial cost function
print(cost(init_params))

0.6767246959339626


#### I'm using Pennylane's built in GradientDescentOptimiser function with stepsize of 0.1 for my circuit. 

In [11]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.1)

# set the number of steps
steps = 2000
# set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.6794910
Cost after step    10:  0.6781395
Cost after step    15:  0.6850222
Cost after step    20:  0.6760754
Cost after step    25:  0.6767289
Cost after step    30:  0.6822979
Cost after step    35:  0.6801791
Cost after step    40:  0.6815602
Cost after step    45:  0.6843474
Cost after step    50:  0.6774930
Cost after step    55:  0.6801847
Cost after step    60:  0.6774226
Cost after step    65:  0.6801847
Cost after step    70:  0.6761347
Cost after step    75:  0.6774564
Cost after step    80:  0.6802184
Cost after step    85:  0.6732993
Cost after step    90:  0.6753775
Cost after step    95:  0.6822476
Cost after step   100:  0.6760416
Cost after step   105:  0.6794981
Cost after step   110:  0.6794868
Cost after step   115:  0.6712004
Cost after step   120:  0.6774226
Cost after step   125:  0.6760359
Cost after step   130:  0.6753436
Cost after step   135:  0.6691563
Cost after step   140:  0.6712047
Cost after step   145:  0.6650132
Cost after ste

Cost after step  1210:  0.0491493
Cost after step  1215:  0.0696105
Cost after step  1220:  0.0858598
Cost after step  1225:  0.0310356
Cost after step  1230:  0.0738193
Cost after step  1235:  0.0711552
Cost after step  1240:  0.0308816
Cost after step  1245:  0.0721468
Cost after step  1250:  0.0567752
Cost after step  1255:  0.0622017
Cost after step  1260:  0.0616783
Cost after step  1265:  0.0603971
Cost after step  1270:  0.0201324
Cost after step  1275:  0.0582428
Cost after step  1280:  0.0611815
Cost after step  1285:  0.0517670
Cost after step  1290:  0.0420035
Cost after step  1295:  0.0428799
Cost after step  1300:  0.0701631
Cost after step  1305:  0.0649249
Cost after step  1310:  0.0411201
Cost after step  1315:  0.0417188
Cost after step  1320:  0.0620713
Cost after step  1325:  0.0615312
Cost after step  1330:  0.0704006
Cost after step  1335:  0.0386329
Cost after step  1340:  0.0732422
Cost after step  1345:  0.0661545
Cost after step  1350:  0.0569429
Cost after ste

### For 1 measurement

In [12]:
dev1.shots = 1
result = circuit(params)
result

array([0., 0., 1., 0.])

### For 10 measurements

In [13]:
dev1.shots = 10
result = circuit(params)
result

array([0. , 0.4, 0.5, 0.1])

### For 100 measurements

In [14]:
dev1.shots = 100
result = circuit(params)
result

array([0.06, 0.47, 0.47, 0.  ])

### For 1000 measurements

In [15]:
dev1.shots = 1000
result = circuit(params)
result

array([0.041, 0.491, 0.446, 0.022])

### Final Circuit parameters


In [16]:
print("Optimized rotation angles: {}".format(params))

Optimized rotation angles: [1.61557108 3.14436579 1.07399146]


Here we see that the first and second parameters are almost close to what we wanted, the first parameter being:
$$\pi/2$$ 
and the second parameter being:
$$\pi$$
However, we are quite far away from the third parameter, which ideally should be 3.14, but came out to be 1.07. That's possibly because of 
- less number of steps
- a comparatively large step size
- Limitations of optimiser's default capabilities

### Here I've also used a different set of stepsize and more steps, and printed the parameters

In [15]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.03)

# set the number of steps
steps = 2500
# set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.6822490
Cost after step    10:  0.6808665
Cost after step    15:  0.6857006
Cost after step    20:  0.6871233
Cost after step    25:  0.6863922
Cost after step    30:  0.6877843
Cost after step    35:  0.6801763
Cost after step    40:  0.6836509
Cost after step    45:  0.6760613
Cost after step    50:  0.6822699
Cost after step    55:  0.6808721
Cost after step    60:  0.6808665
Cost after step    65:  0.6815602
Cost after step    70:  0.6884627
Cost after step    75:  0.6857006
Cost after step    80:  0.6822490
Cost after step    85:  0.6739668
Cost after step    90:  0.6801763
Cost after step    95:  0.6864033
Cost after step   100:  0.6836342
Cost after step   105:  0.6801791
Cost after step   110:  0.6808889
Cost after step   115:  0.6843223
Cost after step   120:  0.6850111
Cost after step   125:  0.6857006
Cost after step   130:  0.6850097
Cost after step   135:  0.6788037
Cost after step   140:  0.6843195
Cost after step   145:  0.6850445
Cost after ste

Cost after step  1210:  0.4424270
Cost after step  1215:  0.4550876
Cost after step  1220:  0.4453510
Cost after step  1225:  0.4517222
Cost after step  1230:  0.4366556
Cost after step  1235:  0.4303262
Cost after step  1240:  0.4402434
Cost after step  1245:  0.4231041
Cost after step  1250:  0.4279004
Cost after step  1255:  0.4141776
Cost after step  1260:  0.4007763
Cost after step  1265:  0.4086584
Cost after step  1270:  0.4164739
Cost after step  1275:  0.4069255
Cost after step  1280:  0.4017187
Cost after step  1285:  0.3845648
Cost after step  1290:  0.3696931
Cost after step  1295:  0.3587389
Cost after step  1300:  0.3749860
Cost after step  1305:  0.3382080
Cost after step  1310:  0.3287986
Cost after step  1315:  0.3282093
Cost after step  1320:  0.3471580
Cost after step  1325:  0.3448759
Cost after step  1330:  0.2902626
Cost after step  1335:  0.3017578
Cost after step  1340:  0.2802887
Cost after step  1345:  0.2619810
Cost after step  1350:  0.2351874
Cost after ste

Cost after step  2415:  0.0649689
Cost after step  2420:  0.0602943
Cost after step  2425:  0.0532831
Cost after step  2430:  0.0683594
Cost after step  2435:  0.0535331
Cost after step  2440:  0.0419467
Cost after step  2445:  0.0529329
Cost after step  2450:  0.0716294
Cost after step  2455:  0.0695282
Cost after step  2460:  0.0430131
Cost after step  2465:  0.0479412
Cost after step  2470:  0.0745905
Cost after step  2475:  0.0563367
Cost after step  2480:  0.0505742
Cost after step  2485:  0.0697747
Cost after step  2490:  0.0490717
Cost after step  2495:  0.0584063
Cost after step  2500:  0.0450597
Optimized rotation angles: [0.63207815 3.14166705 1.59412968]


In [16]:
dev1.shots = 1
result = circuit(params)
result

array([0., 1., 0., 0.])

In [17]:
dev1.shots = 10
result = circuit(params)
result

array([0.2, 0.5, 0.3, 0. ])

In [18]:
dev1.shots = 100
result = circuit(params)
result

array([0.03, 0.44, 0.51, 0.02])

In [19]:
dev1.shots = 1000
result = circuit(params)
result

array([0.057, 0.461, 0.467, 0.015])

### How I made sure it always produces the state |01> + |10> and not |01> - |10>

The answer is, I chose the initial parameters as [pi/2 pi pi] which gives me the state |01> + |10> and not |01> - |10>